In [191]:
import gurobipy as gp

In [257]:
import numpy as np
import numpy.random as npr
N = 25 #25 50 75 100
v = np.zeros(N)
# v = npr.uniform(0.5, 1.5, N)
for i in range(N):
    utility = npr.randint(0, 2)*i*0.05 + npr.rand()*0.1
    v[i] = max(round(utility, 2), 0.01)
print(v)

[0.02 0.01 0.17 0.09 0.07 0.01 0.01 0.43 0.43 0.07 0.05 0.02 0.67 0.69
 0.74 0.78 0.87 0.92 0.9  1.03 0.08 0.01 1.14 0.01 1.21]


In [258]:
def profit_mnl(p, S):
    global v
    pi = 0
    Vj_S = 0
    num = 0
    for i in S:
        num += p[i]*v[i]
        Vj_S += v[i]
    pi += num/(1+Vj_S)
    return pi

In [259]:
def nested_by_price_mnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [260]:
def find_best_option_mnl(p, S):
    global v, N
    current_best = profit_mnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mnl(p, newS)
    return current_best, bestS, flag

def greedy_mnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [261]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [272]:
n_simulations = 100
avg_price = 12

#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psim[k] = abs(npr.normal(avg_price, avg_price, N))
    # psim[k] = npr.uniform(avg_price - math.sqrt(3*avg_price), avg_price + math.sqrt(3*avg_price), N)
    psim[k] = npr.exponential(avg_price, N)
    psim[k] = np.round(psim[k], 2)
    psim[k] = sorted(psim[k], reverse=True)

In [273]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = nested_by_price_mnl(psim[k])
    end_time = time.time()

    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
    
avg_time_nested = st.mean(times_nested)

In [274]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []

for k in range(n_simulations):
    
    start_time = time.time()
    profit, bestS = greedy_mnl(psim[k])
    end_time = time.time()
    
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)

avg_time_greedy = st.mean(times_greedy)

In [275]:
# counting how many instances are different
count = 0
different_results_greedy = []

for k in range(n_simulations):
    if bestS_nested[k] != bestS_greedy[k]:
        count += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_nested[k])
        print(bestS_nested[k], bestS_greedy[k], profits_nested[k], profits_greedy[k])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16] 11.761278688524587 11.666485900216923


In [276]:
if different_results_greedy != []:
    print('DR =  ' + str(count))
    print('Min = ' + str(min(different_results_greedy)))
    print('Mean = ' + str(st.mean(different_results_greedy)))
else:
    print('DR =  0')
    print('Min = NA')
    print('Mean = NA')
print('t = ' + str(100*avg_time_greedy/avg_time_nested))

DR =  1.0
Min = 99.19402651005836
Mean = 99.19402651005836
t = 565.1482264237162
